In [1]:
# Import necessary libraries
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import csv
from opencage.geocoder import OpenCageGeocode

In [2]:
key = '114a46da8f394ca388bb95d41e814ecf'
geocoder = OpenCageGeocode(key)

In [3]:
# Declare input and output CSV files
CSV_file = "Rankings.csv"
output_file = 'output_file.csv'

In [4]:
# Store all cities in a list in City+State format
data = pd.read_csv(CSV_file, encoding="ISO-8859-1")
data['CityState'] = data.City+'--'+data.State
Cities = data['CityState'].tolist()

In [6]:
# Open the output CSV file and Write the header row first
with open(output_file, 'w', newline = '') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['City', 'State', 'Lattitude', 'Longitude', 'Average Salary', 'Job Openings'])
    
    # Webscraping - for each city, get the average salary, number of job openings 
    for CityState in Cities:
        cityname = CityState.split('--')[0]
        statecode= CityState.split('--')[1]
        
        #Get Lat and Long
        results = geocoder.geocode(CityState)
        lat = results[0]['geometry']['lat']
        long = results[0]['geometry']['lng']
        
        try:
            URL = "https://www.indeed.com/career/data-analyst/salaries/" + CityState
            page = requests.get(URL)
            soup = BeautifulSoup(page.text, "html.parser")
            salary = soup.find(name="div", class_="sal-agg-nonbase__average-salary-value" ).text;
        except:
            salary = '0';

        try:
            URL = "https://www.indeed.com/jobs?q=data+analyst&l=" + CityState  + "&explvl=entry_level"
            page = requests.get(URL)
            soup = BeautifulSoup(page.text, "html.parser")
            jobs = soup.find(name="div", id="searchCountPages").text;
            numjobs = jobs.split()[3]
        except:
            numjobs = '0';

        # For each city+state, write a row in the CSV file
        thewriter.writerow([cityname, statecode, lat, long, salary, numjobs])  